<a href="https://colab.research.google.com/github/xdderekchen/NLPs/blob/master/Test_API_IXINBUY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstation of Web API of ixinbuy.com

ixinbuy.com provides Web API for users to access the powerful block-chain based services.

This post has 2 purposes:
 1. Samples for API calls
 2. Testing of API and the webservice


In this version, we will test the following 3 functions
 * **(POST)** http://ixinbuy.com/register
 * **(POST)** http://ixinbuy.com/uploaddata
 * **(GET)**  http://ixinbuy.com/getshareddata

##  1 register

In [5]:
# http://ixinbuy.com/register

import requests
import json

URL = "http://ixinbuy.com/register"

payload = {"device_id":"XD12345"}

#Passing payload as dict
responce = requests.post(URL, data = payload)
print(responce.status_code)

#Passing payload as json string
payload = '{"device_id":"XD12345"}'
responce = requests.post(URL, json = json.loads(payload))
print(responce.status_code)

200
200


## 2.1  uploaddata (single)

In [8]:
# http://ixinbuy.com/uploaddata

import requests
import json

URL = "http://ixinbuy.com/uploaddata"

# data_type": 2 for COVID19 data
payload = {
   "data_type": 2, 
   "data": {
       "device_id": "XD12345",
       "device_data": {
          "province_state": "Guangdong",
          "country_region": "Mainland China",
          "last_update": "2020-03-07T10:43:02",
          "confirmed":1352,
          "deaths": 7,
          "recovered": 1237,
          "latitude":  23.3417,
          "longitude": 113.4244
       }
   }
}

#Passing payload as dict
responce = requests.post(URL, data = payload)
print(responce.status_code)

200


##2.2  uploadeddata (batch)

In here, we will pull real data from John Hopkins and invoke the webservice call.

The data of John Hopkins is at "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports"

def uploaddata(source_file):
   
    

In [48]:
import pandas as pd
import os
import requests
import json

def uploaddata(source_file, URL):
    print("processing: ", source_file)
    df = pd.read_csv(source_file)
    print(df.shape)
    df.rename(columns={'Province/State': 'province_state',
                       'Country/Region': 'country_region',
                       'Last Update'   : 'last_update',
                       'Confirmed':      'confirmed',
                       'Deaths':         'deaths',
                       "Recovered":      "recovered",
                       "Latitude":       "latitude",
                       "Longitude":      "longitude"
                      }, inplace=True)
    
    for i in range(0,  df.shape[0]):
        row = df.iloc[i]
        row_as_dic = row.to_dict()
        payload = {
           "data_type": 2, 
           "data": {
              "device_id": "XD12345",
              "device_data": row_as_dic
           }
        }
        responce = requests.post(URL, data = payload)
        errorCount = 0
        if responce.status_code == 200:
           if (i % 100==0):
              print(i, "", end='', flush=True)
        else:
              errorCount += 1
              print("\nError ", i , " with ", responce.status_code, flush=True)
    print(f"\nProcessed {i+1} of records")
    return (errorCount)
      
       

github_root = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports"
file = os.path.join(github_root, "03-15-2020" + ".csv")

%timeit -n1 -r1 d = uploaddata(file, "http://ixinbuy.com/uploaddata")
print(d)


processing:  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-15-2020.csv
(258, 8)
0 100 200 
Processed 258 of records
1 loop, best of 1: 37.8 s per loop
0


## 3 getshareddata

In [53]:
import pandas as pd
import os
import requests
import json

payload = '''
{
  "data_type": 2,
  "device_id":"XD12345"
}
'''

req = requests.request('GET', 'http://ixinbuy.com/getshareddata', 
                       json = json.loads(payload))

print(req)
data = req.json() 
for i, item in enumerate(data):
  print(i, item)

df = pd.DataFrame(data)
df


<Response [200]>
0 message
1 stack
